<a href="https://colab.research.google.com/github/MS-Oliveira/data_analytics_aula05/blob/main/preparando_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fonte: traduzido e adaptado de https://github.com/GoogleCloudPlatform/training-data-analyst

# Entendendo, explorando e preparando dados

**Objetivos**


1. Ver alguns métodos para resolver valores ausentes e aplicar um deles.
2. Converter uma *feature* para o tipo de dado correto.
3. Renomear uma coluna de *feature* e remover um valor de uma coluna de *feature* que não faz parte da análise.
4. Converter a feature *meanprice* para uma variável categórica.
5. Criar *features* one-hot encoding (para dados categóricos).


## Introdução 

Modelos de *machine learning* só podem consumir valores numéricos.

Dados são ditos "sujos" ou "não limpos" se existem valores de atributos ausentes, contém ruídos ou outliers, tem duplicatas, dados errados e formato não adequado.

Esse notebook apresenta e resolve a maioria dos problemas de dados não limpos. Repare que problemas diferentes podem necessitar de métodos diferentes.

### Importação de bibliotecas

In [ ]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

### Exploração dos dados

Esse dataset é baseado sobre o relatório 
[Vehicle Fuel Type Count by Zip Code](https://data.ca.gov/dataset/vehicle-fuel-type-count-by-zip-code). Foi modificado pelo curso do Google para tornar os dados não limpos para uma representação sintética para propósitos de aprendizagem.


Usar o método read_csv do pandas e verificar as primeiras 5 linhas e as últimas 5 linhas.

In [ ]:
# Fazer a carga do csv
df = pd.read_csv('untidy_vehicle_data_2.csv')

In [ ]:
# Ver as 5 primeiras linhas
df.head()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,NaN,1.0,40726.47
1,10/1/2018,NaN,2014,Gasoline,NaN,Yes,1.0,14600.61
2,NaN,90000,NaN,Gasoline,OTHER/UNK,Yes,NaN,10665.31
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Yes,1.0,31920.35
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.0,27268.07


In [ ]:
# Ver as últimas 5 linhas
df.tail()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
994,6/7/2019,90003,2012,Gasoline,Type_R,Yes,26.0,29740.30
995,6/8/2019,90003,2012,Hybrid Gasoline,OTHER/UNK,Yes,4.0,23112.45
996,6/9/2019,90003,2012,Hybrid Gasoline,Type_Q,Yes,25.0,36798.42
997,6/10/2019,90003,2012,Natural Gas,OTHER/UNK,Yes,1.0,32470.59
998,6/11/2019,90003,2012,Plug-in Hybrid,OTHER/UNK,Yes,3.0,15652.22


### Tipos de dados das colunas

DataFrames podem ter tipos de dados heterogêneos, isso é, uma coluna ser string, outra inteiro e assim por diante.

Arquivos CSV não contém informações sobre tipos de dados a biblioteca Pandas infere os tipos de dados de cada coluna.

Verificar os tipos de dados das colunas, usando .info()

In [ ]:
# Verificar os tipos de dados das colunas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Date        997 non-null    object 
 1   Zip Code    997 non-null    object 
 2   Model Year  997 non-null    object 
 3   Fuel        996 non-null    object 
 4   Make        996 non-null    object 
 5   Light_Duty  996 non-null    object 
 6   Vehicles    996 non-null    float64
 7   meanprice   999 non-null    float64
dtypes: float64(2), object(6)
memory usage: 62.6+ KB


Temos então seis objetos strings e 2 objetos float.

> Bloco com recuo




### Estatísticas de resumo 

Nesse ponto, temos somente uma coluna que contém valores numéricos. Para essas colunas, estamos interessados em várias medidas estatísticas relativas a esses valores. Repare que por ter somente uma *feature* numérica, somente vemos uma estatística de resumo.

In [ ]:
# usar .describe() para obter as estatísticas de resumo de colunas numéricas
df.describe()

,Vehicles,meanprice
count,996.000000,999.000000
mean,72.878514,30312.060811
std,229.696895,11485.268297
min,1.000000,10034.210000
25%,13.000000,20230.065000
50%,23.000000,30644.200000
75%,57.250000,40065.725000
max,3178.000000,49962.760000


### Verificando valores ausentes

Valores ausentes impactam negativamente na qualidade dos dados, cem como podem levar a modelos de *machine learning* fazerem inferências erradas sobre os dados. Valores ausentes podem ser causados por inúmeros fatores, como bits perdidos na transmissão, entrada incorreta de dados ou pelo não preenchimento de um campo. A Pandas reconhece tanto células vazias quando tipos NaN como valores ausentes.

#### Vamos verificar os valores ausentes

Para isso, vamos explorar algumas maneiras de fazer isso.

1ª) Usando .isnull()

In [ ]:
# Usar .isnull()
df.isnull()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
0,False,False,False,False,False,True,False,False
1,False,True,False,False,True,False,False,False
2,True,False,True,False,False,False,True,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
994,False,False,False,False,False,False,False,False
995,False,False,False,False,False,False,False,False
996,False,False,False,False,False,False,False,False
997,False,False,False,False,False,False,False,False


In [ ]:
# usar .sum() ao final de .isnull()
df.isnull().sum()

Date          2
Zip Code      2
Model Year    2
Fuel          3
Make          3
Light_Duty    3
Vehicles      3
meanprice     0
dtype: int64

Podemos selecionar uma ou mais colunas para verificar os valores ausentes

In [ ]:
# Ver quantos valores ausentes somente para a coluna Date
df['Date'].isnull().sum()

2

2º) Usando .isna()

In [ ]:
# Usar .isna()
df.isna()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
0,False,False,False,False,False,True,False,False
1,False,True,False,False,True,False,False,False
2,True,False,True,False,False,False,True,False
3,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...
994,False,False,False,False,False,False,False,False
995,False,False,False,False,False,False,False,False
996,False,False,False,False,False,False,False,False
997,False,False,False,False,False,False,False,False


In [ ]:
# Usar .sum() ao final de .isna()
df.isna().sum()

Date          2
Zip Code      2
Model Year    2
Fuel          3
Make          3
Light_Duty    3
Vehicles      3
meanprice     0
dtype: int64

3º) Podemos estar interessados nos valores não ausentes, nesse caso usamos .notna()

In [ ]:
# Usar .notna() para ver os valores não ausentes
df.notna()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
0,True,True,True,True,True,False,True,True
1,True,False,True,True,False,True,True,True
2,False,True,False,True,True,True,False,True
3,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...
994,True,True,True,True,True,True,True,True
995,True,True,True,True,True,True,True,True
996,True,True,True,True,True,True,True,True
997,True,True,True,True,True,True,True,True


In [ ]:
# Usar .sum() ao final de .notna()
df.notna().sum()

Date          997
Zip Code      997
Model Year    997
Fuel          996
Make          996
Light_Duty    996
Vehicles      996
meanprice     999
dtype: int64

# Vamos sumarizar nossos dados por linha, coluna, *features*, valores únicos e valores ausentes.


In [ ]:
# Mostrar número de linhas
df.shape[0]

999

In [ ]:
# Mostrar número de colunas
df.shape[1]

8

In [ ]:
# Mostrar *features* (ou nomes de colunas). Usar .tolist() ao final.
df.columns.tolist()

['Date',
 'Zip Code',
 'Model Year',
 'Fuel',
 'Make',
 'Light_Duty',
 'Vehicles',
 'meanprice']

In [ ]:
# Mostrar valores únicos .nunique()
df.nunique()

Date          248
Zip Code        6
Model Year     15
Fuel            8
Make           43
Light_Duty      3
Vehicles      210
meanprice     999
dtype: int64

In [ ]:
# Mostrar valores ausentes (usar .values.sum() ao final)
df.isna().values.sum()

18

### Onde podemos melhorar a qualidade de dados?

Lembrando que o objetivo é alimentar algoritmos de *machine learning*.

1. **Questão #1**:  
> **Valores ausentes**:
Cada coluna tem valores ausentes. Na realidade, temos um total de 18 valores ausentes.
2. **Questão #2**: 
> **Tipo de dados de data**:  A coluna Date é mostrada como um objeto e não como um datetime. Além disso, existe um requisito de negócio que a coluna Date seja parseada para ano, mês e dia.
3. **Questão #3**: 
> **Model Year**: Estamos interessados em anos maiores que 2006, não "<2006".
4. **Questão #4**:
> **meanprice**: Queremos transformar os valores em low, mid and high.
5. **Questão #5**:  
> **Colunas categóricas**:  A coluna "Light_Duty" é categorica e tem valores "Yes/No".  Não podemos alimentar um modelo de *machine learning* dessa forma. Precisamos aplicar one-hot encoding nas demais colunas de string.
6. **Questão #6**:
> Os valores de vehicles estão em uma escala bastante ampla, veremos como normalizar (ou padronizar) esses dados.


#### Questão #1:  
##### Resolvendo valores ausentes

A maioria dos algoritmos não aceitam valores ausentes. Para tratar esses valores podemos:

- Remover as linhas com valores ausentes. `.dropna()`.

- Para colunas numéricas: usar a média para preencher os valores.

- Para colunas categóricas: usar a moda para preencher.

Obs.: Método `.fillna()`.

In [ ]:
# Verificar novamente a quantidade de valores ausentes
df.isna().sum()

Date          2
Zip Code      2
Model Year    2
Fuel          3
Make          3
Light_Duty    3
Vehicles      3
meanprice     0
dtype: int64

In [ ]:
# Se quisermos remover os valores ausentes, usar .dropna()
# Não alterar o dataframe ainda
df.dropna()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Yes,1.0,31920.35
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.0,27268.07
5,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,Yes,2.0,35538.39
6,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,WHITE,No,19.0,14666.66
7,10/1/2018,90000,<2006,Battery Electric,OTHER/UNK,No,4.0,29493.14
...,...,...,...,...,...,...,...,...
994,6/7/2019,90003,2012,Gasoline,Type_R,Yes,26.0,29740.30
995,6/8/2019,90003,2012,Hybrid Gasoline,OTHER/UNK,Yes,4.0,23112.45
996,6/9/2019,90003,2012,Hybrid Gasoline,Type_Q,Yes,25.0,36798.42
997,6/10/2019,90003,2012,Natural Gas,OTHER/UNK,Yes,1.0,32470.59


In [ ]:
# Pode usar para uma única coluna. Fazer para Vehicles
df['Vehicles'].dropna()

0       1.0
1       1.0
3       1.0
4      55.0
5       2.0
       ... 
994    26.0
995     4.0
996    25.0
997     1.0
998     3.0
Name: Vehicles, Length: 996, dtype: float64

Na Pandas existe uma expressão lambda, que podemos aplicar a cada elemento.

Uma expressão lambda é uma função sem nome.

Por exemplo, dividir o o valor em Vehicles por 4:

In [ ]:
df['Vehicles'].apply(lambda x: x / 4)

0       0.25
1       0.25
2        NaN
3       0.25
4      13.75
       ...  
994     6.50
995     1.00
996     6.25
997     0.25
998     0.75
Name: Vehicles, Length: 999, dtype: float64

Se aplicarmos o mesmo conceito para o dataframe, o x representa uma coluna.

Para ver o valor mais frequente de uma coluna (moda), podemos usar .value_counts() e pegar seu primeiro elemento:

In [ ]:
# Aplicar value_counts em Zip Code
df['Zip Code'].value_counts()

90002    406
90001    361
90003    213
90000     15
9001       1
na         1
Name: Zip Code, dtype: int64

In [ ]:
# Usar .index[0] para selecionar o primeiro elemento
df['Zip Code'].value_counts().index[0]

'90002'

Podemos então colocar esse valor mais frequente nas observações ausentes das variáveis categóricas:

Primeiro selecionamos as colunas categóricas, aplicamos a função lambda nelas e atribuímos para atualizar para atualizar o dataframe.

Colunas categóricas: 'Date', 'Zip Code', 'Model Year', 'Fuel', 'Make',	'Light_Duty'

Obs.: Date ainda é uma string nesse ponto.

In [ ]:
# Usar .index[0] nas variáveis categóricas
#não funciona ...Pela combinação, tem que aplicar coluna por coluna
df[[ 'Date', 'Zip Code', 'Model Year', 'Fuel', 'Make', 'Light_Duty']].value_counts().index[0]

('7/4/2019', '90003', '2009', 'Gasoline', 'Type_L', 'Yes')

In [ ]:
# agora aplicamos o .apply usando o fillna() com o valor: .value_counts().index[0]
df[[ 'Date', 'Zip Code', 'Model Year', 'Fuel', 'Make', 'Light_Duty']].apply(lambda col: col.fillna(col.value_counts().index[0]) )

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,Yes
1,10/1/2018,90002,2014,Gasoline,OTHER/UNK,Yes
2,10/1/2018,90000,<2006,Gasoline,OTHER/UNK,Yes
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Yes
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No
...,...,...,...,...,...,...
994,6/7/2019,90003,2012,Gasoline,Type_R,Yes
995,6/8/2019,90003,2012,Hybrid Gasoline,OTHER/UNK,Yes
996,6/9/2019,90003,2012,Hybrid Gasoline,Type_Q,Yes
997,6/10/2019,90003,2012,Natural Gas,OTHER/UNK,Yes


In [ ]:
# Podemos agora atribuir ao dataframe
df[[ 'Date', 'Zip Code', 'Model Year', 'Fuel', 'Make', 'Light_Duty']] = df[[ 'Date', 'Zip Code', 'Model Year', 'Fuel', 'Make', 'Light_Duty']].apply(lambda col: col.fillna(col.value_counts().index[0]) )

In [ ]:
# Verificar as 5 primeiras linhas do dataframe
df.head()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,Yes,1.0,40726.47
1,10/1/2018,90002,2014,Gasoline,OTHER/UNK,Yes,1.0,14600.61
2,10/1/2018,90000,<2006,Gasoline,OTHER/UNK,Yes,NaN,10665.31
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Yes,1.0,31920.35
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.0,27268.07


Agora, falta somente preenchermos a coluna Vehicles, que é numérica contínua e para isso, vamos usar a média.

Como é uma coluna só, usamos .fillna() e passamos o primeiro parâmetro com a média dessa coluna:

In [ ]:
df['Vehicles'] = df['Vehicles'].fillna(df['Vehicles'].mean())

In [ ]:
df.head()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice
0,10/1/2018,90000,2006,Gasoline,OTHER/UNK,Yes,1.000000,40726.47
1,10/1/2018,90002,2014,Gasoline,OTHER/UNK,Yes,1.000000,14600.61
2,10/1/2018,90000,<2006,Gasoline,OTHER/UNK,Yes,72.878514,10665.31
3,10/1/2018,90000,2017,Gasoline,OTHER/UNK,Yes,1.000000,31920.35
4,10/1/2018,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.000000,27268.07


Vamos verificar novamente por valores ausentes:

In [ ]:
df.isnull().sum()

Date          0
Zip Code      0
Model Year    0
Fuel          0
Make          0
Light_Duty    0
Vehicles      0
meanprice     0
dtype: int64

#### Questão #2:  
##### Converter a coluna Date para um formato Datetime.

O formato passado é o da origem dos dados. Exemplos:
- 25/02/2020 -> %d/%m/%Y
- 31-03-1997 -> %d-%m-%Y
- 03/31/2009 -> %m/%d/%Y

In [ ]:
# Converter a coluna Date para um datetime .to_datetime
# usando o formato %m%d%Y
df['Date'] = pd.to_datetime(df['Date'], format='%m/%d/%Y')

In [ ]:
# Usar .info() para ver a transformação
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 999 entries, 0 to 998
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Date        999 non-null    datetime64[ns]
 1   Zip Code    999 non-null    object        
 2   Model Year  999 non-null    object        
 3   Fuel        999 non-null    object        
 4   Make        999 non-null    object        
 5   Light_Duty  999 non-null    object        
 6   Vehicles    999 non-null    float64       
 7   meanprice   999 non-null    float64       
dtypes: datetime64[ns](1), float64(2), object(5)
memory usage: 62.6+ KB


Em alguns casos é interessante separar a data em colunas dia, mês e ano.

Por exemplo, identificar padrões sazonais, encontrar períodos incomuns, etc.

Assim como usamos funções de string, temos no pandas funções para string em `.dt`, como `dt.year`, `dt.month` e `dt.day`.

In [ ]:
# Usar dt.year sobre a coluna data
df['Date'].dt.year

0      2018
1      2018
2      2018
3      2018
4      2018
       ... 
994    2019
995    2019
996    2019
997    2019
998    2019
Name: Date, Length: 999, dtype: int64

In [ ]:
# Criar uma coluna year com o ano
df['year'] = df['Date'].dt.year

In [ ]:
# Criar uma coluna month com o mês
df['month'] = df['Date'].dt.month

In [ ]:
# Criar uma coluna day com o dia
df['day'] = df['Date'].dt.day

In [ ]:
# Verificar as colunas criadas olhando as 5 primeiras linhas
df.head()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice,year,month,day
0,2018-10-01,90000,2006,Gasoline,OTHER/UNK,Yes,1.000000,40726.47,2018,10,1
1,2018-10-01,90002,2014,Gasoline,OTHER/UNK,Yes,1.000000,14600.61,2018,10,1
2,2018-10-01,90000,<2006,Gasoline,OTHER/UNK,Yes,72.878514,10665.31,2018,10,1
3,2018-10-01,90000,2017,Gasoline,OTHER/UNK,Yes,1.000000,31920.35,2018,10,1
4,2018-10-01,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.000000,27268.07,2018,10,1


As vezes, queremos agrupar os dados para ter outra forma de analisar os dados, ou para entendermos em outro nível de detalhe.

A pandas tem um método `groupby`, que funciona bastante parecido com o SQL.

In [ ]:
# Agrupar por ano
df.groupby(by ='year')

Com o objeto de groupby criado, aplicamos algumas função de agregação, como first(), last(), sum(), etc.

In [ ]:
# Aplicar first no groupby
df.groupby(by ='year').first()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice,month,day
year,,,,,,,,,,
2018,2018-10-01,90000,2006,Gasoline,OTHER/UNK,Yes,1.0,40726.47,10,1
2019,2019-01-01,90001,2016,Gasoline,Type_G,Yes,18.0,37290.49,1,1


In [ ]:
# Aplicar last no groupby
df.groupby(by ='year').last()

,Date,Zip Code,Model Year,Fuel,Make,Light_Duty,Vehicles,meanprice,month,day
year,,,,,,,,,,
2018,2018-12-31,90002,2018,Hybrid Gasoline,OTHER/UNK,Yes,8.0,43135.62,12,31
2019,2019-06-11,90003,2012,Plug-in Hybrid,OTHER/UNK,Yes,3.0,15652.22,6,11


In [ ]:
# Aplicar sum no groupby
df.groupby(by ='year').sum()

,Vehicles,meanprice,month,day
year,,,,
2018,16622.635542,11701618.42,4192,5968
2019,56183.000000,18580130.33,1634,9020


In [ ]:
# Aplicar mean no groupby
df.groupby(by ='year').mean()

,Vehicles,meanprice,month,day
year,,,,
2018,43.514753,30632.508953,10.973822,15.623037
2019,91.058347,30113.663420,2.648298,14.619125


#### Questão #3

Renomear as colunas de *features*, retirando espaços em branco e transformando letras maísculas em minúsculas.

In [ ]:
# Usar rename (dicionário de valores) com inplace=True
df.rename({'Date':'date',	'Zip Code':'zipcode',	'Model Year':'modelyear',	'Fuel':'fuel',	'Make':'make',	'Light_Duty':'lightduty',	'Vehicles':'vehicles'},axis=1, inplace=True)
# Ver as duas primeiras linhas
df.head(2)

,date,zipcode,modelyear,fuel,make,lightduty,vehicles,meanprice,year,month,day
0,2018-10-01,90000,2006,Gasoline,OTHER/UNK,Yes,1.0,40726.47,2018,10,1
1,2018-10-01,90002,2014,Gasoline,OTHER/UNK,Yes,1.0,14600.61,2018,10,1


Agora, retirar os valores de ano que tenham '<2006'

In [ ]:
# Selecionar os valores com string '<2006'
df[df['modelyear']== '<2006']

,date,zipcode,modelyear,fuel,make,lightduty,vehicles,meanprice,year,month,day
2,2018-10-01,90000,<2006,Gasoline,OTHER/UNK,Yes,72.878514,10665.31,2018,10,1
4,2018-10-01,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,No,55.000000,27268.07,2018,10,1
5,2018-10-01,90000,<2006,Diesel and Diesel Hybrid,OTHER/UNK,Yes,2.000000,35538.39,2018,10,1
6,2018-10-01,90000,<2006,Diesel and Diesel Hybrid,WHITE,No,19.000000,14666.66,2018,10,1
7,2018-10-01,90000,<2006,Battery Electric,OTHER/UNK,No,4.000000,29493.14,2018,10,1
...,...,...,...,...,...,...,...,...,...,...,...
780,2019-03-04,90002,<2006,Gasoline,Type_LL,Yes,47.000000,25700.53,2019,3,4
781,2019-03-05,90002,<2006,Hybrid Gasoline,Type_J,Yes,15.000000,49843.60,2019,3,5
782,2019-03-06,90002,<2006,Hybrid Gasoline,Type_Q,Yes,16.000000,33289.79,2019,3,6
783,2019-03-07,90002,<2006,Natural Gas,OTHER/UNK,Yes,5.000000,48247.13,2019,3,7


In [ ]:
# Atribuir o resultado em df com .copy() 
# Quando alterar quantidade de linhas, usar .copy()
df = df[df['modelyear'] != '<2006'].copy()

In [ ]:
# Usar o value_counts() para confirmar que o valor '<2006' foi removido
df['modelyear'].value_counts()

2007    87
2012    81
2008    79
2011    77
2010    71
2006    70
2015    61
2014    59
2016    57
2017    57
2009    53
2013    52
2018    42
2019     6
Name: modelyear, dtype: int64

#### Questão #4

Converteremos os valores de meanprice para valores low, mid e high.

Para isso iremos discretizar esses valores, conhecido como *binning* ou *bucketization*.

A pandas contém o pd.cut para esse tipo de tarefa.

In [ ]:
# Usar pd.cut com a coluna df['meanprice'], bins=3 e labels=['low', 'mid', 'high']
pd.cut(df['meanprice'], bins=3, labels=['low','mid', 'high'])

0      high
1       low
3       mid
16     high
17      mid
       ... 
994     mid
995     low
996    high
997     mid
998     low
Name: meanprice, Length: 852, dtype: category
Categories (3, object): ['low' < 'mid' < 'high']

In [ ]:
# Substituir a coluna df['meanprice'] com esses valores
df['meanprice'] = pd.cut(df['meanprice'], bins=3, labels=['low','mid', 'high'])

#### Questão #5

**Manipulando colunas categóricas**

A coluna lightduty é uma variável categórica com uma escolha yes/no. Não podemos carregar esse tipo de valores em um modelo de *machine learning*. Precisamos converter as respostas binárias de strings yes/no em inteiros 1/0. Existem vários métodos para fazer isso. Usaremos o método "apply" com uma expressão lambda.


In [ ]:
# Contar número de yes/no na coluna lightduty
df['lightduty'].value_counts()

Yes    771
No      80
--       1
Name: lightduty, dtype: int64

In [ ]:
# Aplicar uma função lambda que transforma Yes em 1 e No em 0
df['lightduty'].apply(lambda x: 0 if x == 'No' else 1)

0      1
1      1
3      1
16     0
17     0
      ..
994    1
995    1
996    1
997    1
998    1
Name: lightduty, Length: 852, dtype: int64

In [ ]:
# Atribuir e verificar que os valores foram alterados
df['lightduty'] = df['lightduty'].apply(lambda x: 0 if x == 'No' else 1)


In [ ]:
df['lightduty'].value_counts()

1    772
0     80
Name: lightduty, dtype: int64

**One-hot encoding**

Algoritmos de *machine Learning* experam vetores de entrada e não variáveis categóricas. Especificamente, eles não pode manipular valores de texto ou de string. Então é frequentemente útil transformar *features* categóricas em vetores.

Um método de transformação é criar variáveis dummy para nossas *features* categóricas. Variáveis Dummy são um conjunto de variáveis binárias (0 ou 1) em que cada uma representa uma única classe de uma *feature* categórica. Simplesmente codificamos a variável categórica como um vetor *one-hot*, isso é, um vetor onde todos os elementos são zero com exceção de 1, que é o *hot*. Com *one-hot encoding*, uma variável categórica se torna um array com amanho igual ao número de escolhas possíveis (número de categorias).

A pandas oferece uma função chamada `get_dummies` para converter uma variável categória em variáveis dummy/indicadoras.

Essa função recebe um parâmetro opcional que chama drop_first. Para algoritmos de machine learning esse parâmetro deve ser True, para evitar um problema matemático. Para outros usos, pode-se deixar esse parâmetro False (que é padrão).

In [71]:
# Usar pd.get_dummies em modelyear
pd.get_dummies(df['modelyear'], drop_first=True) #usar o drop_first=True, quando for utiliza ML

,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,1,0,0
16,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0,0,0,0,0,1,0,0,0,0,0,0,0
995,0,0,0,0,0,1,0,0,0,0,0,0,0
996,0,0,0,0,0,1,0,0,0,0,0,0,0
997,0,0,0,0,0,1,0,0,0,0,0,0,0


In [72]:
# Usar pd.get_dummies em fuel
pd.get_dummies(df['fuel'], drop_first=True)

,Diesel and Diesel Hybrid,Flex-Fuel,Gasoline,Hybrid Gasoline,Natural Gas,Plug-in Hybrid
0,0,0,1,0,0,0
1,0,0,1,0,0,0
3,0,0,1,0,0,0
16,1,0,0,0,0,0
17,1,0,0,0,0,0
...,...,...,...,...,...,...
994,0,0,1,0,0,0
995,0,0,0,1,0,0
996,0,0,0,1,0,0
997,0,0,0,0,1,0


In [73]:
# Usar pd.get_dummies em make
pd.get_dummies(df['make'], drop_first=True)

,Type_A,Type_AA,Type_B,Type_C,Type_D,Type_F,Type_FF,Type_G,Type_GG,Type_H,Type_I,Type_II,Type_J,Type_K,Type_L,Type_M,Type_MM,Type_N,Type_NN,Type_O,Type_OO,Type_P,Type_Q,Type_R,Type_S,Type_T,Type_U,Type_V,Type_W,Type_X,Type_Y,Type_Z
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
16,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [74]:
# Usar pd.get_dummies em zipcode
pd.get_dummies(df['zipcode'], drop_first=True)

,90001,90002,90003,9001,na
0,0,0,0,0,0
1,0,1,0,0,0
3,0,0,0,0,0
16,1,0,0,0,0
17,1,0,0,0,0
...,...,...,...,...,...
994,0,0,1,0,0
995,0,0,1,0,0
996,0,0,1,0,0
997,0,0,1,0,0


In [75]:
# Usar pd.get_dummies em meanprice
pd.get_dummies(df['meanprice'], drop_first=True)

,mid,high
0,0,1
1,0,0
3,1,0
16,0,1
17,1,0
...,...,...
994,1,0
995,0,0
996,0,1
997,1,0


In [76]:
# Criar uma variável dados_dummy e aplicar o pd.get_dummies
# nas colunas ['zipcode','modelyear', 'fuel', 'make', 'meanprice'].
dados_dummy = pd.get_dummies(df[['zipcode','modelyear', 'fuel', 'make', 'meanprice']], drop_first=True)
# Mostrar as 5 primeiras linhas dessa variável


In [77]:
dados_dummy.head()

,zipcode_90001,zipcode_90002,zipcode_90003,zipcode_9001,zipcode_na,modelyear_2007,modelyear_2008,modelyear_2009,modelyear_2010,modelyear_2011,modelyear_2012,modelyear_2013,modelyear_2014,modelyear_2015,modelyear_2016,modelyear_2017,modelyear_2018,modelyear_2019,fuel_Diesel and Diesel Hybrid,fuel_Flex-Fuel,fuel_Gasoline,fuel_Hybrid Gasoline,fuel_Natural Gas,fuel_Plug-in Hybrid,make_Type_A,make_Type_AA,make_Type_B,make_Type_C,make_Type_D,make_Type_F,make_Type_FF,make_Type_G,make_Type_GG,make_Type_H,make_Type_I,make_Type_II,make_Type_J,make_Type_K,make_Type_L,make_Type_M,make_Type_MM,make_Type_N,make_Type_NN,make_Type_O,make_Type_OO,make_Type_P,make_Type_Q,make_Type_R,make_Type_S,make_Type_T,make_Type_U,make_Type_V,make_Type_W,make_Type_X,make_Type_Y,make_Type_Z,meanprice_mid,meanprice_high
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [78]:
# Concatenar pd.concat()  df com dados_dummy (axis=1)
df = pd.concat([df, dados_dummy],axis=1)
# Mostrar as 5 primeiras linhas
df.head()

,date,zipcode,modelyear,fuel,make,lightduty,vehicles,meanprice,year,month,day,zipcode_90001,zipcode_90002,zipcode_90003,zipcode_9001,zipcode_na,modelyear_2007,modelyear_2008,modelyear_2009,modelyear_2010,modelyear_2011,modelyear_2012,modelyear_2013,modelyear_2014,modelyear_2015,modelyear_2016,modelyear_2017,modelyear_2018,modelyear_2019,fuel_Diesel and Diesel Hybrid,fuel_Flex-Fuel,fuel_Gasoline,fuel_Hybrid Gasoline,fuel_Natural Gas,fuel_Plug-in Hybrid,make_Type_A,make_Type_AA,make_Type_B,make_Type_C,make_Type_D,make_Type_F,make_Type_FF,make_Type_G,make_Type_GG,make_Type_H,make_Type_I,make_Type_II,make_Type_J,make_Type_K,make_Type_L,make_Type_M,make_Type_MM,make_Type_N,make_Type_NN,make_Type_O,make_Type_OO,make_Type_P,make_Type_Q,make_Type_R,make_Type_S,make_Type_T,make_Type_U,make_Type_V,make_Type_W,make_Type_X,make_Type_Y,make_Type_Z,meanprice_mid,meanprice_high
0,2018-10-01,90000,2006,Gasoline,OTHER/UNK,1,1.0,high,2018,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2018-10-01,90002,2014,Gasoline,OTHER/UNK,1,1.0,low,2018,10,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2018-10-01,90000,2017,Gasoline,OTHER/UNK,1,1.0,mid,2018,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16,2018-10-09,90001,2006,Diesel and Diesel Hybrid,Type_C,0,16.0,high,2018,10,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17,2018-10-10,90001,2006,Diesel and Diesel Hybrid,OTHER/UNK,0,23.0,mid,2018,10,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


Como alteramos a representação das variáveis categóricas e de data, removemos a representação original.

In [80]:
# Usar df.drop para remover ['date','zipcode','modelyear', 'fuel', 'make', 'meanprice']
df = df.drop(['date','zipcode','modelyear', 'fuel', 'make', 'meanprice'], axis=1)
# Ver as 5 primeiras linhas
df.head()

,lightduty,vehicles,year,month,day,zipcode_90001,zipcode_90002,zipcode_90003,zipcode_9001,zipcode_na,modelyear_2007,modelyear_2008,modelyear_2009,modelyear_2010,modelyear_2011,modelyear_2012,modelyear_2013,modelyear_2014,modelyear_2015,modelyear_2016,modelyear_2017,modelyear_2018,modelyear_2019,fuel_Diesel and Diesel Hybrid,fuel_Flex-Fuel,fuel_Gasoline,fuel_Hybrid Gasoline,fuel_Natural Gas,fuel_Plug-in Hybrid,make_Type_A,make_Type_AA,make_Type_B,make_Type_C,make_Type_D,make_Type_F,make_Type_FF,make_Type_G,make_Type_GG,make_Type_H,make_Type_I,make_Type_II,make_Type_J,make_Type_K,make_Type_L,make_Type_M,make_Type_MM,make_Type_N,make_Type_NN,make_Type_O,make_Type_OO,make_Type_P,make_Type_Q,make_Type_R,make_Type_S,make_Type_T,make_Type_U,make_Type_V,make_Type_W,make_Type_X,make_Type_Y,make_Type_Z,meanprice_mid,meanprice_high
0,1,1.0,2018,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,1.0,2018,10,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,1.0,2018,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16,0,16.0,2018,10,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17,0,23.0,2018,10,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


#### Questão #6.

Alguns algortimos trabalham melhor com valores numéricos (referentes a quantidades) normalizados. Para isso, vamos normalizar a coluna vehicles.

In [81]:
# Calcular a média de df['vehicles']
df['vehicles'].mean()


46.68647712917397

In [82]:
# Subtrair cada valor pela média da coluna
df['vehicles'] - df['vehicles'].mean()


0     -45.686477
1     -45.686477
3     -45.686477
16    -30.686477
17    -23.686477
         ...    
994   -20.686477
995   -42.686477
996   -21.686477
997   -45.686477
998   -43.686477
Name: vehicles, Length: 852, dtype: float64

In [83]:
# Calcular o desvio padrão de df['vehicles']
df['vehicles'].std()

62.02918979289232

In [84]:
# Subtrair cada valor pela média da coluna e dividir pelo desvio padrão
(df['vehicles'] - df['vehicles'].mean()) / df['vehicles'].std()

0     -0.736532
1     -0.736532
3     -0.736532
16    -0.494710
17    -0.381860
         ...   
994   -0.333496
995   -0.688168
996   -0.349617
997   -0.736532
998   -0.704289
Name: vehicles, Length: 852, dtype: float64

In [ ]:
# Atribuir na coluna o resultado da conta acima.
df['vehicles'] = (df['vehicles'] - df['vehicles'].mean()) / df['vehicles'].std()
# Ver as 5 primeiras linhas


In [86]:
df.head()

,lightduty,vehicles,year,month,day,zipcode_90001,zipcode_90002,zipcode_90003,zipcode_9001,zipcode_na,modelyear_2007,modelyear_2008,modelyear_2009,modelyear_2010,modelyear_2011,modelyear_2012,modelyear_2013,modelyear_2014,modelyear_2015,modelyear_2016,modelyear_2017,modelyear_2018,modelyear_2019,fuel_Diesel and Diesel Hybrid,fuel_Flex-Fuel,fuel_Gasoline,fuel_Hybrid Gasoline,fuel_Natural Gas,fuel_Plug-in Hybrid,make_Type_A,make_Type_AA,make_Type_B,make_Type_C,make_Type_D,make_Type_F,make_Type_FF,make_Type_G,make_Type_GG,make_Type_H,make_Type_I,make_Type_II,make_Type_J,make_Type_K,make_Type_L,make_Type_M,make_Type_MM,make_Type_N,make_Type_NN,make_Type_O,make_Type_OO,make_Type_P,make_Type_Q,make_Type_R,make_Type_S,make_Type_T,make_Type_U,make_Type_V,make_Type_W,make_Type_X,make_Type_Y,make_Type_Z,meanprice_mid,meanprice_high
0,1,-0.736532,2018,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,-0.736532,2018,10,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,-0.736532,2018,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
16,0,-0.494710,2018,10,9,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
17,0,-0.381860,2018,10,10,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [87]:
df['vehicles'].mean()

1.4333865341404604e-17

In [88]:
df['vehicles'].std()

1.0000000000000009

## Conclusão

Trabalhamos alguns conceitos pra melhorar a qualidade dos dados com o objetivo desses dados alimentarem algoritmos de machine learning*. Resolvemos:

- Valores ausentes;
- Convertemos tipos de dados para os corretos;
- Renomeamos colunas de features;
- Removemos um valor não desejável de features;
- Transformamos uma variável contínua em categórica;
- Criamos variáveis *one-hot encoding*;
- Convertemos as features temporais para representações significativas;
- Normalizamos uma variável numérica.